In [2]:
from __future__ import print_function
from IPython.display import HTML, display
def go_board(s):
    stones = {x:i%2 for i,x in enumerate(map(eval, s))}
    def pos(i):
        return int(7+(220-6.5)*i/8)
    imgstr = "<img src='img/go%s.png' width='23px' height='23px' style='position: absolute; top: %spx; left: %spx;margin-top: 0' />"
    display(HTML("""<div style="width: 250px; height:250px;position: relative">
   <img src='img/Blank_Go_board_9x9.png' width="100%" height="100%" style="position: relative;left: 0;top: 0;margin-top: 0"/>
   """ + "\n".join(imgstr%('white' if c else 'black', pos(i), pos(j)) for (i,j),c in stones.items()) +"</div>"))

先看看 python 跑的時間

In [6]:
%%timeit -r 1 -n 1
rtn = !python2 go/go.py
go_board(rtn)

1 loops, best of 1: 11.6 s per loop


In [5]:
%%timeit -r 1 -n 1
rtn = !python3 go/go.py
go_board(rtn)

1 loops, best of 1: 13.1 s per loop


換成 pypy 看看

In [7]:
%%timeit -r 1 -n 1
rtn = !pypy go/go.py
go_board(rtn)

1 loops, best of 1: 8.92 s per loop


用 Cython 來編譯同樣的檔案    
下面 cython 將檔案直接編譯成可執行檔的方式

In [8]:
%cd --quick_prototyping_for_speed
%cd go
!cython --embed go.py
!gcc -O2 -o go go.c `python2-config --cflags --ldflags`
!ls -l
%cd ..

/home/tjw/src/cythonup/module/quick_prototyping_for_speed
/home/tjw/src/cythonup/module/quick_prototyping_for_speed/go
總計 1204
-rwxr-xr-x 1 tjw users 591031  7月 22 17:29 go
-rw-r--r-- 1 tjw users 606812  7月 22 17:29 go.c
-rw-r--r-- 1 tjw users   3521  7月 22 12:36 go.pxd
-rw-r--r-- 1 tjw users  13972  7月 22 17:26 go.py
-rw-r--r-- 1 tjw users    165  7月 22 12:36 Makefile
-rw-r--r-- 1 tjw users    254  7月 22 12:36 README.md
/home/tjw/src/cythonup/module/quick_prototyping_for_speed


執行看看

In [20]:
%%timeit -r 1 -n 1
rtn = !go/go
go_board(rtn)

1 loops, best of 1: 598 ms per loop


速度大約是 20 倍， 所以可以讓我們多跑幾步

In [21]:
%%timeit -r 1 -n 1
rtn = !go/go 10
go_board(rtn)

1 loops, best of 1: 5.47 s per loop


因為 cython 會吃 [go.pxd](/edit/go/go.pxd) 來使用型態定義，所以表面上是一樣的原始碼，其實 cython 偷偷多知道很多資訊。